In [1]:
import xarray 
import os
import netCDF4
import numpy as np
import matplotlib.pyplot as plt
import imdlib as imd
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import scipy.io
import matplotlib.image as mpimg
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

In [2]:
x_train = pd.read_csv(r"C:\Users\ratho\Desktop\Software Group Project - III\Climate Data Modelling\Our Work\CSV Excel Data\SSTA Category 1901-2010.csv",index_col=0)
y_train = pd.read_csv(r"C:\Users\ratho\Desktop\Software Group Project - III\Climate Data Modelling\Our Work\CSV Excel Data\Rainfall Cluster Number 1901-2010.csv",index_col=0)
x_test = pd.read_csv(r"C:\Users\ratho\Desktop\Software Group Project - III\Climate Data Modelling\Our Work\CSV Excel Data\SSTA Category 2011-2022.csv",index_col=0)
y_test = pd.read_csv(r"C:\Users\ratho\Desktop\Software Group Project - III\Climate Data Modelling\Our Work\CSV Excel Data\Rainfall Cluster Number 2011-2022.csv",index_col=0)

In [3]:
from sklearn import tree
model = tree.DecisionTreeClassifier(random_state=42)
model.fit(x_train,y_train)

DecisionTreeClassifier(random_state=42)

In [4]:
y_pred = model.predict(x_test)
y_pred

array([3, 4, 8, 3, 3, 3, 8, 3, 3, 3, 3, 6], dtype=int64)

In [5]:
y_array=y_test.values.ravel()
y_array

array([0, 4, 0, 6, 1, 3, 6, 3, 0, 4, 3, 8], dtype=int64)

In [6]:
accuracy_score(y_pred,y_array)

0.3333333333333333

In [7]:
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LinearRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis


In [8]:
model_params = {
    'svm': {
        'model': svm.SVC(decision_function_shape='ovo', gamma='auto'),
        'params': {
            'C': [1, 10, 20],
            'kernel': ['rbf', 'linear'],
            'degree': [2, 3, 4],
            'coef0': [0.0, 0.1, 0.5],
            'probability': [True, False],
        }
    },
    'random_forest': {
        'model': RandomForestClassifier(),
        'params': {
            'n_estimators': [1, 5, 10, 20, 50],
            'criterion': ['gini', 'entropy'],
            'max_depth': [None, 10, 20, 30],
            'min_samples_split': [2, 5, 10],
            'min_samples_leaf': [1, 2, 4],
        }
    },
    'logistic_regression': {
        'model': LogisticRegression(solver='liblinear', multi_class='auto'),
        'params': {
            'C': [1, 5, 10],
            'penalty': ['l1', 'l2'],
        }
    },
    'naive_bayes_gaussian': {
        'model': GaussianNB(),
        'params': {}
    },
    'naive_bayes_multinomial': {
        'model': MultinomialNB(),
        'params': {}
    },
    'decision_tree': {
        'model': DecisionTreeClassifier(random_state=42),
        'params': {
            'criterion': ['gini', 'entropy'],
            'max_depth': [None, 10, 20, 30],
            'min_samples_split': [2, 5, 10],
            'min_samples_leaf': [1, 2, 4],
        }
    },
    'linear_regression': {
        'model': LinearRegression(),
        'params': {}
    },
    'linear_discriminant_analysis': {
        'model': LinearDiscriminantAnalysis(),
        'params': {}
    },
}


In [9]:
y_train_array=y_train.values.ravel()
y_train_array

array([2, 6, 4, 5, 1, 1, 3, 8, 8, 4, 2, 3, 3, 3, 2, 0, 0, 5, 3, 6, 3, 7,
       3, 4, 7, 8, 3, 3, 3, 3, 3, 3, 8, 3, 1, 7, 3, 4, 2, 3, 1, 3, 3, 8,
       8, 3, 4, 2, 3, 3, 2, 1, 7, 3, 3, 8, 3, 3, 0, 3, 0, 1, 3, 4, 6, 6,
       3, 2, 2, 0, 7, 5, 8, 6, 0, 8, 8, 3, 6, 7, 4, 2, 0, 6, 6, 2, 5, 0,
       3, 3, 4, 3, 3, 0, 3, 4, 3, 4, 2, 4, 1, 5, 8, 6, 3, 8, 4, 1, 6, 8],
      dtype=int64)

In [10]:
from sklearn.model_selection import GridSearchCV
import pandas as pd
scores = []

for model_name, mp in model_params.items():
    clf =  GridSearchCV(mp['model'], mp['params'], cv=5, return_train_score=False)
    clf.fit(x_train, y_train_array)
    y_pred = clf.predict(x_test)
    y_pred = np.ceil(y_pred)
    y_test_array= y_test.values.ravel()
    y_test_array
    scores.append({
        'model': model_name,
        'best_score': clf.best_score_,
        'accuracy_score': accuracy_score(y_pred,y_test_array),
        'best_params': clf.best_params_,
    })
    
df = pd.DataFrame(scores,columns=['model','best_score','accuracy_score','best_params'])
df

,model,best_score,accuracy_score,best_params
0,svm,0.300000,0.250000,"{'C': 1, 'coef0': 0.0, 'degree': 2, 'kernel': ..."
1,random_forest,0.354545,0.000000,"{'criterion': 'gini', 'max_depth': 10, 'min_sa..."
2,logistic_regression,0.290909,0.250000,"{'C': 1, 'penalty': 'l1'}"
3,naive_bayes_gaussian,0.218182,0.166667,{}
4,naive_bayes_multinomial,0.318182,0.166667,{}
5,decision_tree,0.254545,0.083333,"{'criterion': 'gini', 'max_depth': None, 'min_..."
6,linear_regression,-0.208417,0.166667,{}
7,linear_discriminant_analysis,0.236364,0.250000,{}
